In [1]:
#import required modules
from bs4 import BeautifulSoup
import ast
import pandas as pd
import re
import requests
from datetime import datetime

# Define Stock Price Functions 

In [2]:
def get_stock_price(ticker):
    # pass a ticker name to i3investor website url
    url = "https://klse.i3investor.com/servlets/stk/chart/{}.jsp". format(ticker)
    # get response from the site and extract the price data
    response = requests.get(url, headers={'User-Agent':'test'})
    soup = BeautifulSoup(response.content, "html.parser")
    script = soup.find_all('script')
    data_tag = script[19].contents[0]
    chart_data = ast.literal_eval(re.findall('\[(.*)\]', data_tag.split(';')[0])[0])
  
    # tabulate the price data into a dataframe
    chart_df = pd.DataFrame(chart_data, columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    
    # convert timestamp into readable date
    chart_df['Date'] = chart_df['Date'].apply(lambda x: \
        datetime.utcfromtimestamp(int(x)/1000).strftime('%Y-%m-%d'))
    
    return chart_df

# Input Stock Ticker

In [3]:
price_chart_df = get_stock_price("MAYBANK")
price_chart_df

,Date,Open,High,Low,Close,Volume
0,2016-10-17,7.62,7.65,7.59,7.60,4312200
1,2016-10-18,7.60,7.71,7.59,7.71,5671900
2,2016-10-19,7.70,7.72,7.67,7.70,3731500
3,2016-10-20,7.69,7.71,7.60,7.60,3896400
4,2016-10-21,7.61,7.69,7.59,7.69,6852600
...,...,...,...,...,...,...
1219,2021-10-11,8.06,8.19,8.06,8.16,7430400
1220,2021-10-12,8.17,8.29,8.13,8.23,10727500
1221,2021-10-13,8.28,8.35,8.24,8.29,10062000
1222,2021-10-14,8.30,8.31,8.25,8.28,4068600


# Create Technical Indicators

In [4]:
def add_EMA(price, day):
   return price.ewm(span=day).mean()

# call function by passing Close prices and EMA period
price_chart_df['EMA18'] = add_EMA(price_chart_df['Close'], 18)
price_chart_df['EMA50'] = add_EMA(price_chart_df['Close'], 50)
price_chart_df['EMA100'] = add_EMA(price_chart_df['Close'], 100)
price_chart_df['EMA200'] = add_EMA(price_chart_df['Close'], 200)

price_chart_df

,Date,Open,High,Low,Close,Volume,EMA18,EMA50,EMA100,EMA200
0,2016-10-17,7.62,7.65,7.59,7.60,4312200,7.600000,7.600000,7.600000,7.600000
1,2016-10-18,7.60,7.71,7.59,7.71,5671900,7.658056,7.656100,7.655550,7.655275
2,2016-10-19,7.70,7.72,7.67,7.70,3731500,7.673618,7.671322,7.670664,7.670333
3,2016-10-20,7.69,7.71,7.60,7.60,3896400,7.652039,7.652408,7.652465,7.652485
4,2016-10-21,7.61,7.69,7.59,7.69,6852600,7.661406,7.660540,7.660275,7.660139
...,...,...,...,...,...,...,...,...,...,...
1219,2021-10-11,8.06,8.19,8.06,8.16,7430400,8.119025,8.154070,8.157855,8.127554
1220,2021-10-12,8.17,8.29,8.13,8.23,10727500,8.130706,8.157048,8.159283,8.128573
1221,2021-10-13,8.28,8.35,8.24,8.29,10062000,8.147474,8.162262,8.161872,8.130180
1222,2021-10-14,8.30,8.31,8.25,8.28,4068600,8.161424,8.166879,8.164211,8.131670
